# LIDOQA: LIDO Quality Assessment

## Introduction

Heterogeneity of asset management systems, export worflows, cataloguing standards, objects results in heterogeneity of LIDO data.
Still, some general rules can be formulated that make LIDO data better.

Some basic guidelines for 'good' LIDO-Data:

- good LIDOs describe objects carefully und comprehensively
- good LIDO is both human and machine-readable
- good LIDO uses controlled vocabulary and complies with good practices of LOD (use URIs wherever possible)

In [ ]:
from glob import glob
from lxml import etree
from collections import Counter, defaultdict
from urllib.parse import urlparse
import requests
from matplotlib import pyplot as plt

In [ ]:
etree.__version__

In [ ]:
# LIDO  Namespace für registrieren
NSMAP = {
    'lido' : 'http://www.lido-schema.org'
}

## Ist die LIDO-XML 'schemavalide'?

Für die optimale Verwendung von XML-Dateien müssen diese oftmals einer möglichst standardisierten Struktur folgen.
Diese Struktur kann in einer [XML Schema Definition Language (XSD)](https://www.w3.org/TR/xmlschema11-1/) beschrieben werden.
Auch für das LIDO-Schema werden XSD-Dateien bereitgestellt:

| LIDO-Version | URL |
|:-- |:--- |
| 1.0 | <http://www.lido-schema.org/schema/v1.0/lido-v1.0.xsd> |
| 1.1 | <http://lido-schema.org/schema/v1.1/lido-v1.1.xsd> |

Es empfiehlt sich, generierte XML-Dateien auf Schemavalidität zu prüfen, d.h. sicherzustellen, dass die XML-Datei den Vorgaben aus der XSD-Schemadatei entspricht.

Für die Prüfung bietet sich das [Python-Modul `lxml`](https://lxml.de/2.1/validation.html#xmlschema) an.

### Erstellung eines  `XMLSchema`-Objekts

Zunächst muss die XSD-Datei eingelesen und in ein [`XMLSchema`-Objekt](https://lxml.de/api/lxml.etree.XMLSchema-class.html) umgewandelt werden.

In [ ]:
#lidoxsd = "http://lido-schema.org/schema/v1.1/lido-v1.1.xsd"
lidoxsd = "http://www.lido-schema.org/schema/v1.0/lido-v1.0.xsd"
res = requests.get(lidoxsd)
xmlschema_doc = etree.fromstring(res.content)
xmlschema = etree.XMLSchema(xmlschema_doc)
# Falls das Schema lokal vorliegt, kann der Dateipfad auch über ein Argument 'file' übergeben werden
# xmlschema = etree.XMLSchema(file="lido-v1.0.xsd")

### Validierung

Für die Validierung an sich gibt es nun mehrere Möglichkeiten.
Das zu validierende XML-Dokument muss dabei jedoch zunächste immer mit `etree.parse(<XML-DATEINAME>)` (oder wenn die XML-Datei als String vorliegt: `etree.fromstring(<XML-STRING>)`) geparset, d.h. zu einem `ElementTree`-Objekt umgewandelt werden.

Im Folgenden sollen alle LIDO-Dateien des `lido`-Ordners auf Schemavalidität geprüft werden.

Gezeigt werden verschiedene Methoden der `XMLSchema`-Klasse.

In [ ]:
lidos = glob('lido/*xml')

#### Überprüfung mit `.validate()`

Die Methode `.validate()` liefert einen Boole'schen Wahrheitswert (`True` bei validem LIDO /`False` bei nicht validem LIDO) zurück.

Eine Liste fehlerhafter Dateien ließe sich also folgendermaßen erzeugen (nähere Informationen zu den Fehlern bietet die `.error_log`-Property des `XMLSchema`-Objekts):

In [ ]:
invalidLidos = []
validLidos = []
for LIDO in lidos:
    tree = etree.parse(LIDO)
    if xmlschema.validate(tree) is False:
        invalidLidos.append(LIDO)
    else:
        validLidos.append(LIDO)

#### Überprüfung mit Fehlermeldung

Während `.validate()` einen Wahrheitswert zurückgibt, führen `.assert_` und `.assertValid` zu Fehlern (`AssertionError` bzw. `DocumentInvalid`), die ggf. über `try` und `except` abgefangen werden müssen.
Die Fehlermeldungen enthalten auch nähere Informationen zum jeweiligen Fehler.
Diese sind auch im `error_log` gespeichert.

In [ ]:
for LIDO in lidos:
    tree = etree.parse(LIDO)
    try:
        xmlschema.assert_(tree)
        #xmlschema.assertValid(tree)
    except Exception as e:
        print(e)

### Interpretation des Validierungsergebnisses

Die Interpretation der Fehlermeldungen der Validierung ist nicht trivial.
Sie setzt eine genauere Auseinandersetzung mit der XSD-Datei bzw. der Spezifikation (LIDO [v.1.0](http://www.lido-schema.org/schema/v1.0/lido-v1.0-specification.pdf)/[v.1.1](https://lido-schema.org/schema/v1.1/lido-v1.1.html)) voraus.

#### Eine kurze Leseanleitung für die LIDO-XSD-Datei

In [ ]:
NSMAP.update({'xsd': 'http://www.w3.org/2001/XMLSchema'})

In [ ]:
for ct in xmlschema_doc.findall('.//xsd:complexType', NSMAP):
    print(ct.tag,ct.attrib)

In [ ]:
types = []
for seq in xmlschema_doc.findall('.//xsd:sequence', NSMAP):
    for x in seq.iter():
        types.append(x.attrib.get('type'))

In [ ]:
Counter(types)

`sequence`-Knoten enhalten `element`-Knoten.
Diese müssen in der angegebenen Reihenfolge erscheinen.
Das Attribut `name` gibt den Namen des Knotens im LIDO-XML-an.


In [ ]:
types = []
for LIDO in lidos:
    print(LIDO)
    tree = etree.parse(LIDO)
    for elem in tree.iter():
        print(elem.attrib)
        if elem.attrib.get('{http://www.lido-schema.org}type'):
            types.append(elem.attrib.get('{http://www.lido-schema.org}type'))

In [ ]:
Counter(types).most_common()

In [ ]:
for LIDO in lidos:
    print(LIDO)
    tree = etree.parse(LIDO)
    for elem in tree.iter():
        #print(elem.attrib)
        if elem.attrib.get('{http://www.lido-schema.org}type'):
            print(f">>> Parent: {elem.find('...').tag}")
            print(f"Tag:\t{elem.tag}\nAttrib:\t{elem.attrib.get('{http://www.lido-schema.org}type')}\nText:\t{elem.text}")
            input()

In [ ]:
conceptIDs = []
for LIDO in lidos:
    tree = etree.parse(LIDO)
    conceptIDs.extend([x.text for x in tree.findall('//lido:conceptID', NSMAP)])

In [ ]:
Counter(conceptIDs).most_common()

In [ ]:
check_urls = {}

In [ ]:
for c in Counter(conceptIDs).most_common():
    if not c[0] in check_urls and not "vocab.getty" in c[0]:
        try:
            res = requests.get(c[0])
            status = res.status_code
            check_urls[c[0]] = status
            print(c[0], status)
        except Exception as e:
            print(e)

In [ ]:
Counter(check_urls.values())

In [ ]:
invalid_urls = []
for k,v in check_urls.items():
    if v != 200:
        print(k,v)
        invalid_urls.append(k)

In [ ]:
from collections import Counter

In [ ]:
invalid_url_dict = defaultdict(list)

In [ ]:
for LIDO in lidos:
    tree = etree.parse(LIDO)
    recID = tree.find('//lido:lidoRecID', NSMAP).text
    for x in tree.findall('//lido:conceptID', NSMAP):
        if x.text in invalid_urls:
            print(recID,LIDO,x.text)
            invalid_url_dict[x.text].append(LIDO)

In [ ]:
import json
with open('output/invalidConceptURLS.json', 'w') as OUT:
    json.dump(invalid_url_dict, OUT)


In [ ]:
invalid = []
for LIDO in lidos:
    ID = LIDO.replace('lido/','').replace('.xml','')
    tree = etree.parse(LIDO)
    print(type(tree))
    instit = tree.find('//lido:repositorySet//lido:legalBodyName/lido:appellationValue', NSMAP).text
    #print(tree.find('//lido:roleActor...', NSMAP))
    if xmlschema.validate(tree) is False:
        log = xmlschema.error_log
        for x in log:
            invalid.append({
                'ID' : ID,
                'line' : x.line,
                'path' : x.path,
                'message' : x.message
            })
    #print(dir(xmlschema))
    #print(xmlschema.validate(tree))
    #print(dir(xmlschema.error_log))
    #print(xmlschema.error_log.last_error)
    
    #try:
    #    xmlschema.assertValid(tree)    
    #except Exception as e:
    #    print(e)
    #    invalid.append(instit)

In [ ]:
with open('LIDO_error_log.csv','w') as OUT:
    writer = csv.DictWriter(OUT, fieldnames = list(invalid[0].keys()))
    writer.writeheader()
    writer.writerows(invalid)

In [ ]:
xmlschema.error_log

## Eregnisse sammeln


In [ ]:
eventtypes = []
for LIDO in lidos:
    tree = etree.parse(LIDO)
    for event in tree.findall('//lido:eventType/lido:term', NSMAP):
        eventtypes.append(event.text)
        

In [ ]:
Counter(eventtypes)

In [ ]:
sources = []
for LIDO in lidos:
    tree = etree.parse(LIDO)
    sources.extend([x.text for x in tree.findall('//*[@lido:source]', NSMAP)])

In [ ]:
c = Counter(sources).most_common()

In [ ]:
sourcesFreq = Counter([urlparse(s).netloc for s in sources]).most_common()[:-20:-1]

In [ ]:
sourcesFreq

In [ ]:
controlled_vocabularies = defaultdict(set)
for LIDO in lidos:
    tree = etree.parse(LIDO)
    for x in tree.findall('//*[@lido:source]', NSMAP):
        controlled_vocabularies[x.attrib.get('{http://www.lido-schema.org}source')].add(x.tag)

In [ ]:
for LIDO in lidos:
    tree = etree.parse(LIDO)
    hit = tree.find('//*[@lido:source="Wikipedia"]', NSMAP)
    if hit is not None:
        print(hit.tag,hit.text,hit.attrib)
        print(LIDO)
        input()


In [ ]:
Counter([r.tag for r in results])

In [ ]:
Counter([r.attrib.get('{http://www.lido-schema.org}source') for r in results if r.tag == "{http://www.lido-schema.org}conceptID" ])

In [ ]:
for r in results:
    print(r.attrib.get('{http://www.lido-schema.org}source'))
    print(r.text)
    print(r.tag)

In [ ]:
sources = defaultdict(set)

In [ ]:
for r in results:
    sources[r.attrib.get('{http://www.lido-schema.org}source')].add(r.tag)

In [ ]:
cv_by_category = defaultdict(set)

for k,v in sources.items():
    for V in v:
        cv_by_category[V].add(k)

In [ ]:
cv_by_category

## Beteiligte Institutionen

Die Datengeber:innen werden im [`recordSource`-Element](http://www.lido-schema.org/schema/v1.1/lido-v1.1.html#recordSource) angegeben.
Dieses Element kann -- und sollte! -- neben dem Namen eine oder mehrere `legalBodyID` enthalten, die die Institution maschinenlesbar identifzieren.
Andernsfalls bleibt lediglich die Identifikation über einen String im `appellationValue` oder eine Web-Adresse (`legalBodyWeblink`)

Die `legalBodyID` ist vom Typ [`identifierComplexType`](http://www.lido-schema.org/schema/v1.1/lido-v1.1.html#identifierComplexType) und enthält i.d.R. das verpflichtende `type`-Attribute sowie das `source`-Element.

Im Folgenden wollen wir uns einen quantitativen Überblick über die Elemente verschaffen und zählen, wie oft die jeweiligen Elemente in den Datensätzen vorkommen.

In [ ]:
contents = []
for LIDO in lidos:
    tree = etree.parse(LIDO)
    for _ in tree.find('//lido:recordSource', NSMAP):
        contents.append(etree.QName(_).localname)

In [ ]:
xs,ys = zip(*Counter(contents).most_common())
plt.bar(xs,ys)
plt.xticks(rotation=30)
plt.show()

Es zeigt sich also, dass die `recordSource`-Elemente den verpflichtenden `legalBodyName` enthalten, aber nur 36k der 50k einen maschinenlesbaren URI.

Zur gezielten Verbesserung der Daten sollen nun die Institutionen gesammelt werden, die (noch) keine `legalBodyID` verzeichnen.

In [ ]:
legalBodiesWithoutID = []
for LIDO in lidos:
    tree = etree.parse(LIDO)
    recordSource = tree.find('//lido:recordSource', NSMAP)
    if recordSource.find('lido:legalBodyID', NSMAP) is None:
        legalBodiesWithoutID.append(recordSource.find('lido:legalBodyName/lido:appellationValue', NSMAP).text)

In [ ]:
for _ in Counter(legalBodiesWithoutID).most_common():
    k,v = _
    print(repr(k),v)

In [ ]:
weblinks_without_ID = []
for LIDO in lidos:
    tree = etree.parse(LIDO)
    recordSource = tree.find('//lido:recordSource', NSMAP)
    if recordSource.find('lido:legalBodyWeblink', NSMAP) is not None and recordSource.find('lido:legalBodyID', NSMAP) is None:
        weblinks_without_ID.append(recordSource.find('lido:legalBodyWeblink', NSMAP).text)

In [ ]:
Counter(weblinks_without_ID)

In [ ]:
sources = []
types = []
for LIDO in lidos:
    tree = etree.parse(LIDO)
    for x in tree.findall('//lido:recordSource/lido:legalBodyID', NSMAP):
        sources.append(x.attrib.get('{http://www.lido-schema.org}source'))
        types.append(x.attrib.get('{http://www.lido-schema.org}type'))

In [ ]:
Counter(sources)

In [ ]:
Counter(types)

## Personendaten

In [ ]:
for LIDO in lidos:
    tree = etree.parse(LIDO)
    for _ in tree.iter():
        print(tree.getpath(_))
    input()

In [ ]:
actor = []
for LIDO in lidos:
    tree = etree.parse(LIDO)
    for _ in tree.findall("//lido:conceptID", NSMAP):
        actor.append(_.text)

In [ ]:
Counter(actor)

Personennamen sind in nameActorSet o.ä. enthalten

In [ ]:
authors_with_id = []
authors_without_id = []
for LIDO in lidos:
    tree = etree.parse(LIDO)
    for actor in tree.findall("//lido:eventActor/lido:actorInRole/lido:actor", NSMAP):
        try:
            actor.find('lido:actorID', NSMAP).text
            authors_with_id.append(actor)
        except:
            authors_without_id.append(actor)

In [ ]:
for a in authors_with_id:
    print(a.tag)
    for x in a:
        print("\t",x.tag)
        for X in x:
             print("\t\t",X.tag)
    print(a.find('.//lido:appellationValue', NSMAP).text)
    for A in a.find('lido:actorID', NSMAP):
        print(A.tag,A.attrib,A.text)
    input()

In [ ]:
Counter((A.text for A in a.iter() for a in authors_with_id))

In [ ]:
Counter((A.text for A in a.iter() for a in authors_without_id))

In [ ]:
for a in authors_with_id:
    for A in a.iter():
        print(A.tag,A.text)
    input()a